In [45]:
import pandas as pd
import ast

In [62]:
trainingdata = pd.read_csv('trainingdata.csv')

In [63]:
trainingdata['categories'] = trainingdata['categories'].apply(ast.literal_eval)
expanded = trainingdata.explode('categories')
one_hot = pd.get_dummies(expanded['categories'])

# Group by the original DataFrame index and take the maximum to condense back to the original shape
resultC = trainingdata.drop('categories', axis=1).join(one_hot.groupby(level=0).max())

In [64]:
trainingdata['mechanics'] = trainingdata['mechanics'].apply(ast.literal_eval)
expanded = trainingdata.explode('mechanics')
one_hot = pd.get_dummies(expanded['mechanics'])

# Group by the original DataFrame index and take the maximum to condense back to the original shape
resultM = trainingdata.drop('mechanics', axis=1).join(one_hot.groupby(level=0).max())

In [65]:
final_result = pd.concat([resultC, resultM], axis=1)

In [67]:
trainingdata = final_result

In [ ]:
#trainingdata.to_csv('cleantraining.csv')

Now for testing data

In [69]:
testingdata = pd.read_csv('testingdata.csv')

In [70]:
testingdata['categories'] = testingdata['categories'].apply(ast.literal_eval)
expanded = testingdata.explode('categories')
one_hot = pd.get_dummies(expanded['categories'])

# Group by the original DataFrame index and take the maximum to condense back to the original shape
resultC = testingdata.drop('categories', axis=1).join(one_hot.groupby(level=0).max())

In [71]:
testingdata['mechanics'] = testingdata['mechanics'].apply(ast.literal_eval)
expanded = testingdata.explode('mechanics')
one_hot = pd.get_dummies(expanded['mechanics'])

# Group by the original DataFrame index and take the maximum to condense back to the original shape
resultM = testingdata.drop('mechanics', axis=1).join(one_hot.groupby(level=0).max())

In [72]:
final_result = pd.concat([resultC, resultM], axis=1)

In [ ]:
#final_result.to_csv('cleantesting.csv')

In [81]:
testingdata = final_result

Need a bit more cleaning done

In [95]:
trainingdata = trainingdata.loc[:,~trainingdata.columns.duplicated()].copy()
testingdata = testingdata.loc[:,~testingdata.columns.duplicated()].copy()

In [97]:
trainmissing =set(testingdata.columns) - set(trainingdata.columns)

In [98]:
testmissing = set(trainingdata.columns) - set(testingdata.columns)

In [99]:
for col in trainmissing:
    trainingdata[col] = False
    
for col in testmissing:
    testingdata[col] = False

In [102]:
set(testingdata.columns)-set(trainingdata.columns)

set()

In [103]:
set(trainingdata.columns)-set(testingdata.columns)

set()

Now I will feature engineer from the description

In [107]:
import re

In [106]:
trainingdata['DescLen']=trainingdata['descriptions'].str.len()
testingdata['DescLen']=testingdata['descriptions'].str.len()

In [113]:
trainingdata['Sentences']=trainingdata['descriptions'].apply(lambda x : len(re.findall(r'[.?!]', x)))
testingdata['Sentences']=testingdata['descriptions'].apply(lambda x : len(re.findall(r'[.?!]', x)))

In [114]:
def unique_word_count(text):
    if pd.isna(text) or text.strip() == "":
        return 0
    # Extract words (alphanumeric sequences)
    words = re.findall(r'\b\w+\b', text.lower())  # Case insensitive
    return len(set(words))  # Count unique words

# Function to calculate special character count
def special_char_count(text):
    if pd.isna(text) or text.strip() == "":
        return 0
    # Count non-alphanumeric and non-whitespace characters
    return len(re.findall(r'[^\w\s]', text))

In [117]:
testingdata['specialChar']=testingdata['descriptions'].apply(special_char_count)
testingdata['uniqueWords']=testingdata['descriptions'].apply(unique_word_count)

trainingdata['specialChar']=trainingdata['descriptions'].apply(special_char_count)
trainingdata['uniqueWords']=trainingdata['descriptions'].apply(unique_word_count)

In [ ]:
#testingdata.to_csv('cleantesting.csv')
#trainingdata.to_csv('cleantraining.csv')